In [18]:
%load_ext autoreload
%autoreload 2

import os
import re
import json
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from argparse import Namespace

import tensorflow as tf
print(tf.__version__)
import tensorflow_io as tfio
print(tfio.__version__)

from tensorflow.keras import layers
from tensorflow.keras import models

import wandb
from wandb.keras import WandbMetricsLogger

from asl.data import InterpolateDataloader
from asl.model import SeparateConvLSTMModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2.11.0
0.31.0


In [19]:
data_path = "../data/tfrecords"


def natural_keys(text):
    ""
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    return [atoi(c) for c in re.split(r'(\d+)', text)]

tfrecords = sorted(glob(f"{data_path}/*.tfrec"), key=natural_keys)

In [20]:
train_tfrecords, valid_tfrecords = tfrecords[:19], tfrecords[19:]
print(len(train_tfrecords)+len(valid_tfrecords))

24


In [21]:
configs = Namespace(
    num_frames=16,
    batch_size=128,
    epochs=20,
    use_wandb=True,
    resizing_interpolation="nearest",
)

In [22]:
dataloader = InterpolateDataloader(configs)
trainloader = dataloader.get_dataloader(train_tfrecords)
validloader = dataloader.get_dataloader(valid_tfrecords, dataloader="valid")

In [24]:
tf.keras.backend.clear_session()
model = SeparateConvLSTMModel(configs).get_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 543, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 tf.compat.v1.gather (TFOpLambd  (None, 16, 40, 3)   0           ['input_1[0][0]']                
 a)                                                                                               
                                                                                                  
 conv_lstm1d (ConvLSTM1D)       (None, 33, 32)       35968       ['tf.compat.v1.gather[0][0]']    
                                                                                              